In [ ]:
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, roc_auc_score
import string
import nltk
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import average_precision_score, precision_recall_curve
from imblearn.over_sampling import SMOTE
import pandas as pd
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc, roc_auc_score
import string
import nltk
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import average_precision_score, precision_recall_curve
from imblearn.over_sampling import SMOTE
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import cross_val_score, cross_val_predict
from nltk.stem.porter import *
from nltk.corpus import stopwords
import numpy as np
from sklearn.model_selection import GridSearchCV

In [ ]:
reviews_df = pd.read_csv("Hotel_Reviews.csv")
reviews_df.head(3)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968


In [ ]:
print("Duplicated rows before: ",reviews_df.duplicated().sum())
reviews_df.drop_duplicates(inplace=True)
print("Duplicated rows after: ",reviews_df.duplicated().sum())

Duplicated rows before:  526
Duplicated rows after:  0


In [ ]:
reviews_df.dropna(inplace=True)
reviews_df.isna().sum()

Hotel_Address                                 0
Additional_Number_of_Scoring                  0
Review_Date                                   0
Average_Score                                 0
Hotel_Name                                    0
Reviewer_Nationality                          0
Negative_Review                               0
Review_Total_Negative_Word_Counts             0
Total_Number_of_Reviews                       0
Positive_Review                               0
Review_Total_Positive_Word_Counts             0
Total_Number_of_Reviews_Reviewer_Has_Given    0
Reviewer_Score                                0
Tags                                          0
days_since_review                             0
lat                                           0
lng                                           0
dtype: int64

In [ ]:
print('No Positive values before:', end=" ")
print(reviews_df['Positive_Review'].value_counts().get('No Positive', 0))
print('No Negative values before:', end=" ")
print(reviews_df['Negative_Review'].value_counts().get('No Negative', 0))

reviews_df['Negative_Review']= reviews_df['Negative_Review'].replace("No Negative" , "")
reviews_df['Positive_Review']= reviews_df['Positive_Review'].replace("No Positive" , "")

reviews_df['Review'] = reviews_df['Negative_Review'].astype (str).str.cat (reviews_df['Positive_Review'], sep=' ')
reviews_df.head(5)

No Positive values before: 35695
No Negative values before: 126902


,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,Review
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,I am so angry that i made this post available...
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,No real complaints the hotel was great great...
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Rooms are nice but for elderly a bit difficul...
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968,My room was dirty and I was afraid to walk ba...
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968,You When I booked with your company on line y...


In [ ]:
reviews_df_1 = reviews_df.loc[:, ['Review','Reviewer_Score']]
reviews_df_1.head(5)

,Review,Reviewer_Score
0,I am so angry that i made this post available...,2.9
1,No real complaints the hotel was great great...,7.5
2,Rooms are nice but for elderly a bit difficul...,7.1
3,My room was dirty and I was afraid to walk ba...,3.8
4,You When I booked with your company on line y...,6.7


In [ ]:
def multiclass(x):
  if x<=3.5:
    return 0
  elif (x<=6.5 and x>=5.5):
    return 1
  elif x==10:
    return 2
  else:
    return 3

reviews_df_1["Review_Type"] = reviews_df_1["Reviewer_Score"].apply(multiclass)
reviews_df_1.drop(reviews_df_1[reviews_df_1.Review_Type == 3].index, inplace=True)
reviews_df_1.head(5)

,Review,Reviewer_Score,Review_Type
0,I am so angry that i made this post available...,2.9,0
7,Apart from the price for the brekfast Everyth...,10.0,2
8,Even though the pictures show very clean room...,6.5,1
10,Nothing all great Rooms were stunningly dec...,10.0,2
11,6 30 AM started big noise workers loading woo...,5.8,1


In [ ]:
reviews_df_1['Review_Type'].value_counts()

2    115148
1     27587
0      6628
Name: Review_Type, dtype: int64

In [ ]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [ ]:
def ConvertToLower(text):
    temp = text.lower()
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)

    return temp

stop_words = stopwords.words('english')
reviews_df_1['Review'] = reviews_df_1['Review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
reviews_df_1['Review'] = reviews_df_1['Review'].apply(ConvertToLower)
reviews_df_1.head()

,Review,Reviewer_Score,Review_Type
0,i angry made post available via possible sites...,2.9,0
7,apart price brekfast everything good good loca...,10.0,2
8,even though pictures show clean rooms actual r...,6.5,1
10,nothing great rooms stunningly decorated reall...,10.0,2
11,6 30 am started big noise workers loading wood...,5.8,1


In [ ]:
  review_features=reviews_df_1.copy()
  review_features=review_features[['Review']].reset_index(drop=True)
  review_features.head()

,Review
0,i angry made post available via possible sites...
1,apart price brekfast everything good good loca...
2,even though pictures show clean rooms actual r...
3,nothing great rooms stunningly decorated reall...
4,6 30 am started big noise workers loading wood...


In [ ]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(review_features)):
    review = re.sub('[^a-zA-Z]', ' ', review_features['Review'][i])
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stop_words]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus[3]

'noth great room stunningli decor realli spaciou top build pictur room true beauti build kept modernis brilliantli also bath love big invit great coupl restaur menu bit pricey load littl eateri place nearbi within walk distanc tram stop centr minut walk away stop centr amsterdam would recommend hotel anyon unbeliev well price'

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000,ngram_range=(2,2))
X= tfidf_vectorizer.fit_transform(review_features['Review'])

In [ ]:
y = reviews_df_1['Review_Type']

In [ ]:
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X, y)

In [ ]:
print(f'Resampled dataset shape {y_res.value_counts()}')

Resampled dataset shape 0    115148
2    115148
1    115148
Name: Review_Type, dtype: int64


In [ ]:
reviews_df_1['Review_Type'].value_counts()

2    115148
1     27587
0      6628
Name: Review_Type, dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.25, random_state=0)

In [ ]:
from sklearn.metrics import accuracy_score

def model_predict(clf, X_train,y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    predictions = clf.predict(X_test)
    print_report(clf, X_test, y_test)

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report

def print_report(clf, X_test, y_test):
    y_pred = clf.predict(X_test)
    report = metrics.classification_report(y_test, y_pred)
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_test, y_pred)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std

RForest = RandomForestClassifier (n_estimators = 100, random_state = 42)
RForest.fit(X_train,y_train)
y_pred = RForest.predict(X_test)
print("Classification Report:\n",classification_report(y_test, y_pred))

cv = KFold(n_splits=4, random_state=1, shuffle=True)
scores = cross_val_score(RForest, X_test, y_test, scoring='accuracy', cv=cv)
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.97      0.90     28786
           1       0.88      0.85      0.86     28731
           2       0.94      0.80      0.86     28844

    accuracy                           0.88     86361
   macro avg       0.88      0.88      0.88     86361
weighted avg       0.88      0.88      0.88     86361

Accuracy: 0.802 (0.003)


In [44]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(random_state=0)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print("Classification Report:\n",classification_report(y_test, y_pred))
scores = cross_val_score(logreg, X_test, y_test, scoring='accuracy', cv=cv)
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Classification Report:
               precision    recall  f1-score   support

           0       0.70      0.85      0.76     28786
           1       0.74      0.65      0.70     28731
           2       0.90      0.81      0.85     28844

    accuracy                           0.77     86361
   macro avg       0.78      0.77      0.77     86361
weighted avg       0.78      0.77      0.77     86361



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Accuracy: 0.753 (0.004)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.naive_bayes import MultinomialNB
MNaiveBayes = MultinomialNB()
MNaiveBayes.fit(X_train,y_train)
y_pred = MNaiveBayes.predict(X_test)
print("Classification Report:\n",classification_report(y_test, y_pred))

In [ ]:
#from sklearn.linear_model import LogisticRegression

#param_grid = {'C': np.logspace(-4, 4, 50),
#             'penalty':['l1', 'l2']}
#clf = GridSearchCV(LogisticRegression(random_state=0), param_grid,cv=5, verbose=0,n_jobs=-1)
#best_model = clf.fit(X_train,y_train)
#print(best_model.best_estimator_)
#print("The mean accuracy of the model is:",best_model.score(X_test,y_test))

In [ ]:
from sklearn.svm import LinearSVC
LSVC = LinearSVC(random_state=42)
LSVC.fit(X_train,y_train)
y_pred = LSVC.predict(X_test)
print("Classification Report:\n",classification_report(y_test, y_pred))

In [ ]:
pip install xgboost

In [ ]:
from xgboost import XGBClassifier



Xgb = XGBClassifier()
Xgb.fit(X_train,y_train)
y_pred = Xgb.predict(X_test)
print("Classification Report:\n",classification_report(y_test, y_pred))

In [ ]:
#Distribution of Reviews Score/ Reviews Type - !Done!
#Year vs Months graph https://www.kaggle.com/code/mbkinaci/detailed-eda-and-xgboost-89-accuracy #Average Score for Each Month Done!
#Best period of activity of the Hotel (F.I: Date Since Review <20 Days) Done!
#Word Count Bad/Good Reviews Done!
#Competitors analysis by reviews/Services Done!
#Top 5 Hotels Wordcount (to see which are good thing in hotel) Done!
#Mentioned/Not Mentioned Graph Not to Be Done!
#Colorful Graph in Text Classification 📑 😀🙁 (Classic ML+NN+Tf-HUB) / Review Sentimet Distribution Done!


# Compare Pretrained Model Result with our Model Result (?) // To-Do
# HotelsMap / Different colors for average score (bad mid good)
# Develope a Chatbot for Customer Service/Other Works IDEA(?)